<left>
    <img src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Logo_SYGNET.png" width="90" alt="cognitiveclass.ai logo">
</left>

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/2d/Tensorflow_logo.svg" width="200" alt="cognitiveclass.ai logo">
</center>




# A.I. Neural Training
## Part II - Polynomial Chaos Expansion

Postprocessing the simulation results from ZSOIL for Neural Network training.

## Objectives

By conducting this thorough data analysis, we'll gain a deeper understanding of the dataset and the underlying physical processes. This will not only help in building a better NN model but also provide valuable insights into the soil subsidence phenomenon in our simulation. These insights can guide feature selection, inform model architecture decisions, and improve interpretability of the NN's results.

*   Data Science with Python
*   Statistics

<h3>Table of Contents</h3>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li><a href="#III. Feature Engineering"><b>II. Feature Engineering</b></a></li>
        <li><a href="#- Derived and Cumulative Variables">- Correlation and Causality</a></li>
    </ul>
</div>

<hr>

Installing TensorFlow

In [2]:
!pip install tensorflow==2.9.0
!pip install numpy==1.21.4
!pip install statsmodels

Importing Libraries

In [6]:
import time
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.integrate import dblquad
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from statsmodels.stats.stattools import omni_normtest, jarque_bera
from scipy.stats import skew, kurtosis

print(scipy.__version__)

1.7.3


Loading the <i>AI_Dataset</i>

In [5]:
# Load and preprocess the data
subsidence_data = pd.read_csv('PIM_Subsidence.csv')
print(subsidence_data.columns.tolist())

['TIME', 'SF', 'PUSHOVER LABEL', 'PUSHOVER LAMBDA', 'PUSHOVER U-CTRL', 'ARC LENGTH STEP', 'ARC LENGTH U-NORM', 'ARC LENGTH LOAD FACTOR', 'NR', 'X', 'Y', 'Displacement-X', 'Displacement-Y', 'Rotation-Z', 'Total head-', 'Residual Force-X', 'Residual Force-Y', 'Residual Heat flux-Z', 'Solid-Velocity-X', 'Solid-Velocity-Y', 'Solid-Acceleration-X', 'Solid-Acceleration-Y', 'Unnamed: 23', 'TIME.1', 'SF.1', 'PUSHOVER LABEL.1', 'PUSHOVER LAMBDA.1', 'PUSHOVER U-CTRL.1', 'ARC LENGTH STEP.1', 'ARC LENGTH U-NORM.1', 'ARC LENGTH LOAD FACTOR.1', 'ELEM.', 'GP', 'Z', 'Eff.Stress-XY', 'Eff.Stress-XZ', 'Eff.Stress-YZ', 'Tot.Stress-XZ', 'Tot.Stress-YZ', 'Strain-XX', 'Strain-YY', 'Strain-XY', 'Strain-ZZ', 'Strain-XZ', 'Strain-YZ', 'Strain-11', 'Strain-33', 'Strain-J2^1/2', 'Stress level', 'Saturation', 'Fluid velocity-X', 'Fluid velocity-Y', 'Fluid velocity-ABS', 'Pc', 'Pore pressure', 'alpha*(S*pF+<dpF_undr>)', 'Temperature', 'Unnamed: 60', 'Gradient_DisplacementY_X', 'Gradient_StressYY_X', 'Gradient_Stre

# Test Metamodel

Normalization

In [ ]:
from sklearn.preprocessing import StandardScaler
subsidence_scaler = StandardScaler()
data['Subsidence'] = subsidence_scaler.fit_transform(data[['Subsidence']])

# Define a mapping from string to numeric values
mapping = {'elastic': 0, 'plastic': 1, 'brittle': 2}

# Apply the mapping to the column
data['Stress_Regime'] = data['Stress_Regime'].map(mapping)

# Handle any missing or unmapped values
data['Stress_Regime'] = data['Stress_Regime'].fillna(-1)  # or any other default value


Define Chebyshev Polynomials

Define Chebyshev polynomials of the first kind

In [ ]:
# Example: Define Chebyshev polynomial of degree 3
T3 = chebyt(3)
x = np.linspace(-1, 1, 100)
y = T3(x)

# Plotting the polynomial
import matplotlib.pyplot as plt
plt.plot(x, y, label='T3(x)')
plt.legend()
plt.show()

Define Jacobi Polynomials

Specify the parameters (\alpha) and (\beta), which define the type of Jacobi polynomial.

In [ ]:
from scipy.special import jacobi

# Example: Define Jacobi polynomial of degree 3 with alpha=0.5 and beta=0.5
J3 = jacobi(3, 0.5, 0.5)
y_jacobi = J3(x)

# Plotting the polynomial
plt.plot(x, y_jacobi, label='J3(x)')
plt.legend()
plt.show()

Construct the PCE Metamodel

To construct the PCE metamodel, you need to create a basis of polynomials and then use them to approximate your model. Here’s a simplified example:

In [ ]:
from sklearn.linear_model import LinearRegression

# Generate sample data
X = np.random.uniform(-1, 1, (100, 1))
y = np.sin(X).ravel() + 0.1 * np.random.randn(100)

# Define polynomial basis functions
def chebyshev_basis(x, degree):
    return np.array([chebyt(d)(x) for d in range(degree + 1)]).T

def jacobi_basis(x, degree, alpha, beta):
    return np.array([jacobi(d, alpha, beta)(x) for d in range(degree + 1)]).T

# Combine basis functions
degree = 3
alpha, beta = 0.5, 0.5
X_chebyshev = chebyshev_basis(X, degree)
X_jacobi = jacobi_basis(X, degree, alpha, beta)
X_combined = np.hstack((X_chebyshev, X_jacobi))

# Fit the PCE model
model = LinearRegression()
model.fit(X_combined, y)

# Predict using the PCE model
y_pred = model.predict(X_combined)

# Plot the results
plt.scatter(X, y, label='Data')
plt.plot(X, y_pred, label='PCE Model', color='red')
plt.legend()
plt.show()

You may need to adjust the degree of the polynomials or the parameters (\alpha) and (\beta) for Jacobi polynomials to better capture the characteristics of your data.

Additional Resources

[SciPy Documentation on Chebyshev Polynomials](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.chebyt.html) 

[SciPy Documentation on Jacobi Polynomials](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.chebyt.html) 
    
    


In [7]:
!pip install chaospy
!pip install openturns

In [8]:
import chaospy as cp
import openturns as ot
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# openTURNS Approach:

Optionally map PIM_Subsidence by StandardScaler

In [10]:
from sklearn.preprocessing import StandardScaler
subsidence_scaler = StandardScaler()
subsidence_data['PIM_Subsidence'] = subsidence_scaler.fit_transform(subsidence_data[['PIM_Subsidence']])

# Define a mapping from string to numeric values
mapping = {'elastic': 0, 'plastic': 1, 'brittle': 2}

# Apply the mapping to the column
subsidence_data['Stress_Regime'] = subsidence_data['Stress_Regime'].map(mapping)

# Handle any missing or unmapped values
subsidence_data['Stress_Regime'] = subsidence_data['Stress_Regime'].fillna(-1)  # or any other default value

Tailoring the 3 Datasets

In [7]:
# Load data
lstm_train_predictions_df = pd.read_csv('train_predictions.csv')
lstm_test_predictions_df = pd.read_csv('test_predictions.csv')
subsidence_data = pd.read_csv('PIM_Subsidence.csv')

# Convert 'Stress Regime' categorical column to numerical using one-hot encoding
subsidence_data = pd.get_dummies(subsidence_data, columns=['Stress_Regime'])

# Convert predictions to numeric arrays
lstm_train_predictions = lstm_train_predictions_df.values.flatten().astype(float)
lstm_test_predictions = lstm_test_predictions_df.values.flatten().astype(float)

# Calculate the difference in length
difference = len(subsidence_data) - len(lstm_train_predictions)

# Pad the smaller dataset by repeating the last value
lstm_train_predictions_padded = np.pad(lstm_train_predictions, (0, difference), 'edge')
print("Shape of padded lstm_train_predictions:", lstm_train_predictions_padded.shape)

# Calculate the difference in length
difference = len(lstm_train_predictions_padded) - len(lstm_test_predictions)

# Pad the smaller dataset by repeating the last value
lstm_test_predictions_padded = np.pad(lstm_test_predictions, (0, difference), 'edge')
lstm_test_predictions=lstm_test_predictions_padded
lstm_train_predictions=lstm_train_predictions_padded
# Check the new shape
print("Shape of padded lstm_test_predictions:", lstm_test_predictions_padded.shape)
print(f"Shape of lstm_train_predictions: {lstm_train_predictions.shape}")
print(f"Shape of lstm_test_predictions: {lstm_train_predictions.shape}")

Shape of padded lstm_train_predictions: (15973,)
Shape of padded lstm_test_predictions: (15973,)
Shape of lstm_train_predictions: (15973,)
Shape of lstm_test_predictions: (15973,)


Splitting the Test Samples

In [8]:
# Assuming 'PIM_Subsidence' is the target column
X = subsidence_data.drop(columns=['PIM_Subsidence']).values  # Features
y = subsidence_data['PIM_Subsidence'].values  # Target

# Calculate test_size as a proportion of the total data length
test_size = len(lstm_test_predictions) / len(subsidence_data)

# Determine the split index based on the length of the training predictions
split_index = len(lstm_train_predictions)

# Manually split the data
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

# Ensure the test size is reasonable (e.g., 20% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Similarly split the LSTM predictions to match the train/test split
lstm_train_predictions_split, lstm_test_predictions_split = train_test_split(
    lstm_train_predictions, test_size=0.2, random_state=42)

# Calculate residuals for training set
train_residuals = y_train - lstm_train_predictions_split.flatten()

# Calculate residuals for test set
test_residuals = y_test - lstm_test_predictions_split.flatten()


# Ensure that the shapes are correct
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


# Verify the shapes
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

# Ensure the length of predictions matches the respective training and test data
#assert len(lstm_train_predictions) == len(y_train), "Mismatch in the length of training predictions"
#assert len(lstm_test_predictions) == len(y_test), "Mismatch in the length of test predictions"

print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of lstm_train_predictions: {lstm_train_predictions.shape}")


Shape of X_train: (12778, 74)
Shape of X_test: (3195, 74)
Shape of y_train: (12778,)
Shape of y_test: (3195,)
Shape of X_train: (12778, 74)
Shape of X_test: (3195, 74)
Shape of y_train: (12778,)
Shape of y_test: (3195,)
Shape of y_train: (12778,)
Shape of lstm_train_predictions: (15973,)


Defining Distributions

In [53]:
# Ensure that X_train is a numeric array
X_train = X_train.astype(float)


# Define the distributions for each input feature
#distributions = [cp.Uniform(np.min(X_train[:, i]), np.max(X_train[:, i])) for i in range(X_train.shape[1])]

# Create the joint distribution for all input features
#joint_distribution = cp.J(*distributions)


# Define a small epsilon value
epsilon = 1e-6

# Define the distributions for each input feature
distributions = [ot.Uniform(np.min(X_train[:, i]), np.max(X_train[:, i]) + epsilon) for i in range(X_train.shape[1])]

# Create the joint distribution for all input features
joint_distribution = ot.ComposedDistribution(distributions)

# Generate a sample from the joint distribution
sample = joint_distribution.getSample(X_train.shape[0])


Defining Polynomial Basis

In [54]:

# Define Chebyshev polynomial basis of order 3
chebyshev_basis = ot.OrthogonalUniVariatePolynomialFamily(ot.ChebychevFactory())

# Define Jacobi polynomial basis of order 3 with alpha=0 and beta=0
jacobi_basis = ot.OrthogonalUniVariatePolynomialFamily(ot.JacobiFactory(0, 0))


Polynomial Chaos

In [15]:
sample = joint_distribution.getSample(X_train.shape[0])

basis_chebyshev = ot.OrthogonalProductPolynomialFactory([ot.ChebychevFactory()] * X_train.shape[1])
adaptive_strategy = ot.FixedStrategy(sample)
projection_strategy = ot.LeastSquaresStrategy()

pce_chebyshev = ot.FunctionalChaosAlgorithm(sample, train_residuals, joint_distribution, adaptive_strategy, projection_strategy)
pce_chebyshev.run()
result_chebyshev = pce_chebyshev.getResult()
metamodel_chebyshev = result_chebyshev.getMetaModel()

basis_jacobi = ot.OrthogonalProductPolynomialFactory([ot.JacobiFactory()] * X_train.shape[1])
pce_jacobi = ot.FunctionalChaosAlgorithm(sample, train_residuals, joint_distribution, adaptive_strategy, projection_strategy)
pce_jacobi.run()
result_jacobi = pce_jacobi.getResult()
metamodel_jacobi = result_jacobi.getMetaModel()
residual_pred_chebyshev = np.array([metamodel_chebyshev(X_test[i, :]) for i in range(X_test.shape[0])])
residual_pred_jacobi = np.array([metamodel_jacobi(X_test[i, :]) for i in range(X_test.shape[0])])

final_pred_chebyshev = lstm_test_predictions.flatten() + residual_pred_chebyshev
final_pred_jacobi = lstm_test_predictions.flatten() + residual_pred_jacobi

mse_chebyshev = mean_squared_error(y_test, final_pred_chebyshev)
mse_jacobi = mean_squared_error(y_test, final_pred_jacobi)

print(f"Mean Squared Error with Chebyshev basis: {mse_chebyshev}")
print(f"Mean Squared Error with Jacobi basis: {mse_jacobi}")

TypeError: Wrong number or type of arguments for overloaded function 'new_FixedStrategy'.
  Possible C/C++ prototypes are:
    OT::FixedStrategy::FixedStrategy()
    OT::FixedStrategy::FixedStrategy(OT::OrthogonalBasis const &,OT::UnsignedInteger const)
    OT::FixedStrategy::FixedStrategy(OT::FixedStrategy const &)


In [27]:
import openturns as ot

# Define the Chebyshev polynomial factory for each feature
chebyshev_basis_factory = ot.ChebychevFactory()

# Define the Jacobi polynomial factory for each feature
jacobi_basis_factory = ot.JacobiFactory(0, 0)

# Combine them into a multivariate polynomial basis
chebyshev_collection = [chebyshev_basis_factory for _ in range(X_train.shape[1])]
jacobi_collection = [jacobi_basis_factory for _ in range(X_train.shape[1])]

# Create the orthogonal product polynomial factories
basis_chebyshev = ot.OrthogonalProductPolynomialFactory(chebyshev_collection)
basis_jacobi = ot.OrthogonalProductPolynomialFactory(jacobi_collection)
# Create a linear enumerate function based on the number of features
enumerate_function = ot.LinearEnumerateFunction(X_train.shape[1])

# Set the desired polynomial degree
total_degree = 3  # Example: total degree of polynomials

# Determine the strata index that corresponds to the desired polynomial degree
strata_index = enumerate_function.getMaximumDegreeStrataIndex(total_degree)

# Calculate the cumulative cardinality up to the desired strata index
basis_size = enumerate_function.getStrataCumulatedCardinal(strata_index)

# Create the fixed strategy for Chebyshev basis
adaptive_strategy_chebyshev = ot.FixedStrategy(basis_chebyshev, basis_size)

# Create the fixed strategy for Jacobi basis
adaptive_strategy_jacobi = ot.FixedStrategy(basis_jacobi, basis_size)
# Projection strategy using Least Squares
projection_strategy = ot.LeastSquaresStrategy()

# Sample from the joint distribution
sample = joint_distribution.getSample(X_train.shape[0])


In [28]:
# Assume `sample_data` and `train_residuals_data` are provided as NumPy arrays
sample = ot.Sample(sample_data)
train_residuals = ot.Sample(train_residuals_data)

# Define the joint distribution (using a multivariate normal as an example)
dimension = sample.getDimension()
marginals = [ot.Normal()] * dimension
copula = ot.IndependentCopula(dimension)
joint_distribution = ot.ComposedDistribution(marginals, copula)

# Define the orthogonal basis and adaptive strategy
polynomials = ot.StandardDistributionPolynomialFactory(ot.Normal())
basis = ot.OrthogonalProductPolynomialFactory([polynomials] * dimension)
adaptive_strategy_chebyshev = ot.FixedStrategy(basis, 10)

# Define the projection strategy using least squares
projection_strategy = ot.LeastSquaresStrategy()

# Construct and run the FunctionalChaosAlgorithm
pce_chebyshev = ot.FunctionalChaosAlgorithm(sample, train_residuals, joint_distribution, adaptive_strategy_chebyshev, projection_strategy)
pce_chebyshev.run()

NameError: name 'sample_data' is not defined

# CHAOSPy approach:

In [ ]:
# Get the PCE result and metamodel
result_chebyshev = pce_chebyshev.getResult()
metamodel_chebyshev = result_chebyshev.getMetaModel()

# Run the PCE algorithm for Jacobi basis
pce_jacobi = ot.FunctionalChaosAlgorithm(sample, train_residuals, joint_distribution, adaptive_strategy_jacobi, projection_strategy)
pce_jacobi.run()

result_jacobi = pce_jacobi.getResult()
metamodel_jacobi = result_jacobi.getMetaModel()
# Predict residuals using the Chebyshev metamodel
residual_pred_chebyshev = np.array([metamodel_chebyshev(X_test[i, :]) for i in range(X_test.shape[0])])

# Combine residual predictions with the original LSTM predictions
final_pred_chebyshev = lstm_test_predictions.flatten() + residual_pred_chebyshev

# Calculate mean squared error for Chebyshev basis
mse_chebyshev = mean_squared_error(y_test, final_pred_chebyshev)
print(f"Mean Squared Error with Chebyshev basis: {mse_chebyshev}")

# Predict residuals using the Jacobi metamodel
residual_pred_jacobi = np.array([metamodel_jacobi(X_test[i, :]) for i in range(X_test.shape[0])])

# Combine residual predictions with the original LSTM predictions
final_pred_jacobi = lstm_test_predictions.flatten() + residual_pred_jacobi

# Calculate mean squared error for Jacobi basis
mse_jacobi = mean_squared_error(y_test, final_pred_jacobi)
print(f"Mean Squared Error with Jacobi basis: {mse_jacobi}")

In [9]:
from scipy.interpolate import interp1d

# Create an interpolation function
interp_func = interp1d(np.arange(train_residuals.size), train_residuals, kind='linear')

# Generate new indices for interpolation
new_indices = np.linspace(0, train_residuals.size - 1, 74 * 12778)

# Interpolate the data
train_residuals_interpolated = interp_func(new_indices).reshape(74, 12778)

print(train_residuals_interpolated.shape)  # Should be (74, 12778)



(74, 12778)


In [10]:
print(X_train.T.shape)
print(train_residuals.shape)

# Repeat the data to fit the new shape
train_residuals_repeated = np.tile(train_residuals, (74, 1))

print(train_residuals_repeated.shape)  # Should be (74, 12778)

(74, 12778)
(12778,)
(74, 12778)


In [52]:
# Fit polynomial chaos expansions
pce_chebyshev = cp.fit_regression(chebyshev_basis, joint_distribution, X_train.T, train_residuals_interpolated)
pce_jacobi = cp.fit_regression(jacobi_basis, joint_distribution, X_train.T, train_residuals_interpolated)

# Predict residuals for the test set
residual_pred_chebyshev = pce_chebyshev(X_test.T)
residual_pred_jacobi = pce_jacobi(X_test.T)

# Calculate final predictions
final_pred_chebyshev = lstm_test_predictions.flatten() + residual_pred_chebyshev
final_pred_jacobi = lstm_test_predictions.flatten() + residual_pred_jacobi

# Calculate mean squared errors
mse_chebyshev = mean_squared_error(y_test, final_pred_chebyshev)
mse_jacobi = mean_squared_error(y_test, final_pred_jacobi)

print(f"Mean Squared Error with Chebyshev basis: {mse_chebyshev}")
print(f"Mean Squared Error with Jacobi basis: {mse_jacobi}")
print(X_train.T.shape)
print(train_residuals.shape)

AssertionError: 

In [58]:
# Ensure the shapes are correct
print(X_train.T.shape)  # Should be (74, 12778)
print(train_residuals_interpolated.shape)  # Should be (74, 12778)

# Fit polynomial chaos expansions
try:
    pce_chebyshev = cp.fit_regression(chebyshev_basis, joint_distribution, X_train.T, train_residuals_interpolated)
    pce_jacobi = cp.fit_regression(jacobi_basis, joint_distribution, X_train.T, train_residuals_interpolated)
except AssertionError as e:
    print(f"AssertionError: {e}")


(74, 12778)
(74, 12778)
AssertionError: 


In [ ]:
import chaospy as cp
import openturns as ot
import numpy as np

epsilon = 1e-6  # Small value to avoid zero range
distributions = [ot.Uniform(np.min(X_train[:, i]), np.max(X_train[:, i]) + epsilon) for i in range(X_train.shape[1])]
joint_distribution = ot.ComposedDistribution(distributions)

cp_distributions = cp.J(*[cp.Uniform(np.min(X_train[:, i]), np.max(X_train[:, i]) + epsilon) for i in range(X_train.shape[1])])
chebyshev_basis = cp.orth_ttr(3, cp_distributions)  # Order 3 Chebyshev basis
jacobi_basis = cp.orth_ttr(3, cp_distributions)  # Order 3 Jacobi basis with alpha=0 and beta=0

try:
    pce_chebyshev = cp.fit_regression(chebyshev_basis, cp_distributions, X_train.T, train_residuals_interpolated)
    pce_jacobi = cp.fit_regression(jacobi_basis, cp_distributions, X_train.T, train_residuals_interpolated)
except AssertionError as e:
    print(f"AssertionError: {e}")

    

chaospy.orth_ttr name is to be deprecated; Use chaospy.expansion.stieltjes instead


In [ ]:
# Fit polynomial chaos expansions
pce_chebyshev = cp.fit_regression(chebyshev_basis, joint_distribution, X_train.T, train_residuals_interpolated)
pce_jacobi = cp.fit_regression(jacobi_basis, joint_distribution, X_train.T, train_residuals_interpolated)

# Predict residuals for the test set
residual_pred_chebyshev = pce_chebyshev(X_test.T)
residual_pred_jacobi = pce_jacobi(X_test.T)

# Calculate final predictions
final_pred_chebyshev = lstm_test_predictions.flatten() + residual_pred_chebyshev
final_pred_jacobi = lstm_test_predictions.flatten() + residual_pred_jacobi

# Calculate mean squared errors
mse_chebyshev = mean_squared_error(y_test, final_pred_chebyshev)
mse_jacobi = mean_squared_error(y_test, final_pred_jacobi)

print(f"Mean Squared Error with Chebyshev basis: {mse_chebyshev}")
print(f"Mean Squared Error with Jacobi basis: {mse_jacobi}")
print(X_train.T.shape)
print(train_residuals.shape)